In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import stats
import plotly.express as px


In [4]:
path = r"C:\Users\lnae0002\Desktop\autolamella\autolamella\log\g"

df_sample, df_history, df_shift, df_steps  = stats.calculate_statistics_dataframe(path)


--------------------------------------------------------------------------------
STATUS | 01-brief-lark | Setup | LAMELLA_ADDED
{'lamella': '01-brief-lark', 'stage': 'Setup', 'step': 'LAMELLA_ADDED', 'timestamp': 1683601721.0, 'step_n': 0}
STATUS | 01-brief-lark | Setup | MILLING_FIDUCIAL
{'lamella': '01-brief-lark', 'stage': 'Setup', 'step': 'MILLING_FIDUCIAL', 'timestamp': 1683601723.0, 'step_n': 1}
STATUS | 01-brief-lark | FiducialMilled | FIDUCIAL_MILLED_SUCCESSFULLY
{'lamella': '01-brief-lark', 'stage': 'FiducialMilled', 'step': 'FIDUCIAL_MILLED_SUCCESSFULLY', 'timestamp': 1683601744.0, 'step_n': 2}
STATUS | 02-grand-mouse | Setup | LAMELLA_ADDED
{'lamella': '02-grand-mouse', 'stage': 'Setup', 'step': 'LAMELLA_ADDED', 'timestamp': 1683601777.0, 'step_n': 3}
STATUS | 02-grand-mouse | Setup | MILLING_FIDUCIAL
{'lamella': '02-grand-mouse', 'stage': 'Setup', 'step': 'MILLING_FIDUCIAL', 'timestamp': 1683601779.0, 'step_n': 4}
STATUS | 02-grand-mouse | FiducialMilled | FIDUCIAL_MILLED_S

In [8]:
display(df_steps)
fig = px.scatter(df_steps, x="step_n", y="timestamp", color="stage", symbol='lamella')
fig.show()



,lamella,stage,step,timestamp,step_n
0,01-brief-lark,Setup,LAMELLA_ADDED,1.683602e+09,0
1,01-brief-lark,Setup,MILLING_FIDUCIAL,1.683602e+09,1
2,01-brief-lark,FiducialMilled,FIDUCIAL_MILLED_SUCCESSFULLY,1.683602e+09,2
3,02-grand-mouse,Setup,LAMELLA_ADDED,1.683602e+09,3
4,02-grand-mouse,Setup,MILLING_FIDUCIAL,1.683602e+09,4
...,...,...,...,...,...
100,05-alert-prawn,RegularCut,BEAM_ALIGNMENT,1.683605e+09,100
101,05-alert-prawn,RegularCut,BEAM_ALIGNMENT,1.683605e+09,101
102,05-alert-prawn,RegularCut,BEAM_ALIGNMENT,1.683605e+09,102
103,05-alert-prawn,RegularCut,MILLING_TRENCH PolishingCut,1.683605e+09,103


,lamella,stage,step,timestamp,step_n,delta
0,01-brief-lark,Setup,LAMELLA_ADDED,1.683602e+09,0,NaN
1,01-brief-lark,Setup,MILLING_FIDUCIAL,1.683602e+09,1,2.0
2,01-brief-lark,FiducialMilled,FIDUCIAL_MILLED_SUCCESSFULLY,1.683602e+09,2,21.0
3,02-grand-mouse,Setup,LAMELLA_ADDED,1.683602e+09,3,33.0
4,02-grand-mouse,Setup,MILLING_FIDUCIAL,1.683602e+09,4,2.0
...,...,...,...,...,...,...
100,05-alert-prawn,RegularCut,BEAM_ALIGNMENT,1.683605e+09,100,0.0
101,05-alert-prawn,RegularCut,BEAM_ALIGNMENT,1.683605e+09,101,0.0
102,05-alert-prawn,RegularCut,BEAM_ALIGNMENT,1.683605e+09,102,0.0
103,05-alert-prawn,RegularCut,MILLING_TRENCH PolishingCut,1.683605e+09,103,0.0


In [12]:
# calculate difference in timestamp between rows
df_steps['delta'] = df_steps['timestamp'].diff()

display(df_steps)

px.bar(df_steps, x="lamella", y="delta", color="step", facet_col="stage")

,lamella,stage,step,timestamp,step_n,delta
0,01-brief-lark,Setup,LAMELLA_ADDED,1.683602e+09,0,NaN
1,01-brief-lark,Setup,MILLING_FIDUCIAL,1.683602e+09,1,2.0
2,01-brief-lark,FiducialMilled,FIDUCIAL_MILLED_SUCCESSFULLY,1.683602e+09,2,21.0
3,02-grand-mouse,Setup,LAMELLA_ADDED,1.683602e+09,3,33.0
4,02-grand-mouse,Setup,MILLING_FIDUCIAL,1.683602e+09,4,2.0
...,...,...,...,...,...,...
100,05-alert-prawn,RegularCut,BEAM_ALIGNMENT,1.683605e+09,100,0.0
101,05-alert-prawn,RegularCut,BEAM_ALIGNMENT,1.683605e+09,101,0.0
102,05-alert-prawn,RegularCut,BEAM_ALIGNMENT,1.683605e+09,102,0.0
103,05-alert-prawn,RegularCut,MILLING_TRENCH PolishingCut,1.683605e+09,103,0.0


In [3]:
fig = px.bar(df_history, x="stage", y="duration", color="petname", barmode="group")

fig.show()

In [19]:
# display(df_history)
df_history["duration"].mean()

df_sorted = df_history.groupby(["stage", "petname"]).mean()
df_sorted.drop(columns=["start", "end"], inplace=True)
df_sorted.reset_index(inplace=True)
display(df_sorted)


C:\Users\lnae0002\AppData\Local\Temp\ipykernel_18028\550369350.py:4: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,stage,petname,duration
0,FiducialMilled,alert-prawn,21.845123
1,FiducialMilled,brief-lark,21.853174
2,FiducialMilled,fair-bison,21.849651
3,FiducialMilled,firm-bobcat,21.847729
4,FiducialMilled,grand-mouse,21.853731
5,PolishingCut,alert-prawn,0.518324
6,PolishingCut,brief-lark,89.987437
7,PolishingCut,fair-bison,22.878284
8,PolishingCut,firm-bobcat,45.243562
9,PolishingCut,grand-mouse,67.633583
